In [ ]:

%%capture
import os
import site
os.sys.path.insert(0, '/home/schirrmr/code/reversible/reversible2/')
os.sys.path.insert(0, '/home/schirrmr/braindecode/code/braindecode/')
os.sys.path.insert(0, '/home/schirrmr/code/explaining/reversible//')
%cd /home/schirrmr/


%load_ext autoreload
%autoreload 2
import numpy as np
import logging
log = logging.getLogger()
log.setLevel('INFO')
import sys
logging.basicConfig(format='%(asctime)s %(levelname)s : %(message)s',
                     level=logging.INFO, stream=sys.stdout)
import matplotlib
from matplotlib import pyplot as plt
from matplotlib import cm
%matplotlib inline
%config InlineBackend.figure_format = 'png'
matplotlib.rcParams['figure.figsize'] = (12.0, 1.0)
matplotlib.rcParams['font.size'] = 14
import seaborn
seaborn.set_style('darkgrid')

from reversible.sliced import sliced_from_samples

from numpy.random import RandomState

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.autograd import Variable
import numpy as np
import copy
import math

import itertools
from reversible.plot import create_bw_image
import torch as th
from braindecode.torch_ext.util import np_to_var, var_to_np
from reversible.revnet import ResidualBlock, invert, SubsampleSplitter, ViewAs, ReversibleBlockOld
from spectral_norm import spectral_norm
from conv_spectral_norm import conv_spectral_norm

def display_text(text, fontsize=18):
    fig = plt.figure(figsize=(12,0.1))
    plt.title(text, fontsize=fontsize)
    plt.axis('off')
    display(fig)
    plt.close(fig)

In [ ]:
from braindecode.datasets.bbci import BBCIDataset

# we loaded all sensors to always get same cleaning results independent of sensor selection
# There is an inbuilt heuristic that tries to use only EEG channels and that definitely
# works for datasets in our paper
#train_loader = BBCIDataset('/data/schirrmr/schirrmr/HGD-public/reduced/train/13.mat')
#test_loader = BBCIDataset('/data/schirrmr/schirrmr/HGD-public/reduced/test/13.mat')
cnt = BBCIDataset('/data/schirrmr/schirrmr/HGD-public/reduced/train/4.mat',
                 load_sensor_names=['C3', 'C4']).load()
cnt = cnt.drop_channels(['STI 014'])

In [ ]:
from collections import OrderedDict
from braindecode.datautil.trial_segment import create_signal_target_from_raw_mne

marker_def = OrderedDict([('Right Hand', [1]), ('Left Hand', [2],),
                         ('Rest', [3]), ('Feet', [4])])
ival = [500,1500]
from braindecode.mne_ext.signalproc import mne_apply, resample_cnt
from braindecode.datautil.signalproc import exponential_running_standardize

log.info("Resampling train...")
cnt = resample_cnt(cnt, 250.0)
log.info("Standardizing train...")
cnt = mne_apply(lambda a: exponential_running_standardize(a.T ,factor_new=1e-3, init_block_size=1000, eps=1e-4).T,
                     cnt)
cnt = resample_cnt(cnt, 2.0)
train_set = create_signal_target_from_raw_mne(cnt, marker_def, ival)

In [ ]:
x_right = train_set.X[train_set.y == 0]

x_rest = train_set.X[train_set.y == 2]

In [ ]:
def rev_block(n_c, n_i_c):
     return ReversibleBlockOld(
        nn.Sequential(
        nn.Conv2d(n_c // 2, n_i_c,(3,1), stride=1, padding=(1,0),bias=True),
        nn.ReLU(),
            nn.Conv2d(n_i_c, n_c // 2,(3,1), stride=1, padding=(1,0),bias=True)),
         
        nn.Sequential(
        nn.Conv2d(n_c // 2, n_i_c,(3,1), stride=1, padding=(1,0),bias=True),
        nn.ReLU(),
            nn.Conv2d(n_i_c, n_c // 2,(3,1), stride=1, padding=(1,0),bias=True))
    )

In [ ]:
inputs = np_to_var(x_rest[:10,0:1,:,None], dtype=np.float32).cuda() + 4

In [ ]:
plt.figure(figsize=(5,5))
plt.plot(var_to_np(inputs.squeeze()[:,0]), var_to_np(inputs.squeeze()[:,1]), marker='o', ls='')

In [ ]:
from reversible.revnet import SubsampleSplitter, ViewAs
from reversible.util import set_random_seeds
from reversible.revnet import init_model_params
from torch.nn import ConstantPad2d
import torch as th
from conv_spectral_norm import conv_spectral_norm
set_random_seeds(2019011641, True)
feature_model = nn.Sequential(
    SubsampleSplitter(stride=[2,1],chunk_chans_first=False),
    rev_block(2,32),
    rev_block(2,32),
    ViewAs((-1,2,1,1), (-1,2))
)
feature_model.cuda()


from braindecode.torch_ext.util import set_random_seeds
adv_model = nn.Sequential(
    ViewAs((-1,1,2,1), (-1,2)),
    ResidualBlock(
        nn.Sequential(
            nn.Linear(2,100),
            nn.ReLU(),
            nn.Linear(100,2)),
    ),
    ResidualBlock(
        nn.Sequential(
            nn.Linear(2,100),
            nn.ReLU(),
            nn.Linear(100,2)),
    ),
    nn.Linear(2,1)
    )

from spectral_norm import spectral_norm

adv_model.cuda()



from reversible.training import hard_init_std_mean
n_dims = 2
n_clusters = 1
# will be initialized properly later
means_per_cluster = [th.autograd.Variable(th.zeros(n_dims).cuda(), requires_grad=True)
                     for _ in range(n_clusters)]
stds_per_cluster = [th.autograd.Variable(th.ones(n_dims).cuda(), requires_grad=True)
                    for _ in range(n_clusters)]


# set only two dimensions to 1
means_per_cluster[0].data[:] = 0
stds_per_cluster[0].data[:] = 0
stds_per_cluster[0].data[0] = 1
stds_per_cluster[0].data[1] = 1


from copy import deepcopy
optimizer = th.optim.Adam(
                          [
    {'params': list(feature_model.parameters()),
    'lr': 1e-3,
    'weight_decay': 0},], betas=(0,0.9))


optim_adv = th.optim.Adam([{
    'params': adv_model.parameters(),
    'lr': 4e-3, 'weight_decay': 0.00}],#lr 0.0004
                         betas=(0,0.9))

mean = means_per_cluster[0]
std = stds_per_cluster[0]

In [ ]:
from reversible.gaussian import get_gauss_samples
from reversible.revnet import invert 
import pandas as pd


df = pd.DataFrame()
g_loss = np_to_var([np.nan],dtype=np.float32)
g_grad = np.nan
d_loss = np_to_var([np.nan],dtype=np.float32)
d_grad = np.nan

In [ ]:
from torch import autograd
def gradient_penalty(critic, real_data, generated_data):
    batch_size = real_data.size()[0]

    # Calculate interpolation
    alpha = th.rand(batch_size, 1, 1, 1)
    alpha = alpha.expand_as(real_data)
    if real_data.is_cuda:
        alpha = alpha.cuda()
    interpolated = alpha * real_data.data + (1 - alpha) * generated_data.data
    interpolated = interpolated.detach().requires_grad_(True)
    if real_data.is_cuda:
        interpolated = interpolated.cuda()

    # Calculate score of interpolated examples
    score_interpolated = critic(interpolated)

    # Calculate gradients of scores with respect to examples
    gradients = autograd.grad(outputs=score_interpolated, inputs=interpolated,
                           grad_outputs=torch.ones(score_interpolated.size()).cuda() if real_data.is_cuda else torch.ones(
                           prob_interpolated.size()),
                           create_graph=True, retain_graph=True)[0]

    # Derivatives of the gradient close to 0 can cause problems because of
    # the square root, so manually calculate norm and add epsilon
    gradients_norm = torch.sqrt(torch.sum(gradients ** 2, dim=1) + 1e-12)

    # Return gradient penalty
    return (nn.functional.relu(gradients_norm - 1) ** 2).mean()

In [ ]:
n_epochs = 4001
for i_epoch in range(n_epochs):
    samples = get_gauss_samples(20, mean, std)

    inverted = invert(feature_model, samples)

    score_fake = adv_model(inverted)
    if (i_epoch % 10) != 0:
        score_real = adv_model(inputs)
        #d_loss = nn.functional.relu(1.0 - score_real).mean() + nn.functional.relu(
        #            1.0 + score_fake).mean()
        gradient_loss = gradient_penalty(adv_model, inputs, inverted[:(len(inputs))])
        d_loss = -score_real.mean() + score_fake.mean() + gradient_loss
        optim_adv.zero_grad()
        d_loss.backward()
        optim_adv.step()
        d_grad = np.mean([th.sum(p.grad **2).item() for p in adv_model.parameters()])
    else:
        g_loss = -th.mean(score_fake)
        optimizer.zero_grad()
        g_loss.backward()

        optimizer.step()
        g_grad = np.mean([th.sum(p.grad **2).item() for p in feature_model.parameters()])
    samples = get_gauss_samples(200, mean, std)

    inverted = invert(feature_model, samples)
    in_np = var_to_np(inputs).squeeze()
    fake_np = var_to_np(inverted).squeeze()
    import ot

    dist = np.sum(np.abs(in_np[:,None] - fake_np[None]), axis=2)
    match_matrix = ot.emd([],[], dist)
    cost = np.sum(dist * match_matrix)
    
    score_fake = adv_model(inverted)
    score_real = adv_model(inputs)
    wd_dist = th.mean(score_real) - th.mean(score_fake)
    epoch_row = {
    'd_loss': d_loss.item(),
    'g_loss': g_loss.item(),
    'o_real': th.mean(score_real).item(),
    'o_fake': th.mean(score_fake).item(),
    'g_grad': g_grad,
    'd_grad': d_grad,
    'wd_diff': cost - wd_dist.item(),
    'wd_dist': wd_dist.item(),
    'sampled_wd': cost }
    df = df.append(epoch_row, ignore_index=True)

    if i_epoch % (n_epochs // 20) == 0:
        display_text("Epoch {:d}".format(i_epoch))
        display(df.iloc[-5:])
        
        i_std_2, i_std_1 = np.argsort(var_to_np(stds_per_cluster[0]))[::-1][:2]
        """
        feature_a_values = th.linspace(float(mean[i_std_1].data - 2 * std[i_std_1].data),
                               float(mean[i_std_1].data + 2 * std[i_std_1].data), 9)
        feature_b_values = th.linspace(float(mean[i_std_2].data - 2 * std[i_std_2].data),
                               float(mean[i_std_2].data + 2 * std[i_std_2].data), 9)
        image_grid = np.zeros((len(feature_a_values), len(feature_b_values), inputs.shape[2]))

        for i_f_a_val, f_a_val in enumerate(feature_a_values):
            for i_f_b_val, f_b_val in enumerate(feature_b_values):
                this_out = mean.clone()
                this_out.data[i_std_1] = f_a_val
                this_out.data[i_std_2] = f_b_val
                inverted = var_to_np(invert(feature_model, this_out.unsqueeze(0))[0]).squeeze()

                image_grid[i_f_a_val, i_f_b_val] = np.copy(inverted)
        fig, axes = plt.subplots(image_grid.shape[0], image_grid.shape[1],
             sharex=True, sharey=True,
            figsize=(int(image_grid.shape[0] * 2), int(image_grid.shape[1])))
        plt.subplots_adjust(wspace=0, hspace=0)
        for ax, curve  in zip(axes.flatten(), image_grid.reshape(-1, image_grid.shape[-1])):
            ax.plot(curve)
        display(fig)
        plt.close(fig)
        """
        
        fig = plt.figure(figsize=(4,4))
        outs_np = var_to_np(feature_model(inputs))

        plt.plot(outs_np[:,i_std_1],
                 outs_np[:,i_std_2],
                 marker='o', ls='')
        plt.plot(var_to_np(inputs.squeeze()[:,0]), var_to_np(inputs.squeeze()[:,1]),
                 marker='o', ls='')
        for a_in, a_out in zip(var_to_np(inputs.squeeze()), outs_np):
            plt.plot([a_in[0], a_out[0]], [a_in[1], a_out[1]], color='black', lw=1)

        plt.legend(('Input', 'Output'))
        limits = min(plt.xlim()[0], plt.ylim()[0]), max(plt.xlim()[1], plt.ylim()[1])
        plt.xlim(limits)
        plt.ylim(limits)
        plt.title("Encoder output for real data")
        display(fig)
        plt.close(fig)
        fig = plt.figure(figsize=(12,2))
        plt.title("Std of different dimensions")
        plt.plot(np.std(outs_np, axis=0))
        plt.xlabel('Output dimension')
        display(fig)
        plt.close(fig)
        
        samples = get_gauss_samples(200, mean, std)

        inverted = invert(feature_model, samples)

        score_fake = adv_model(inverted).squeeze()
        
        fig = plt.figure(figsize=(5,5))
        max_abs_score = th.max(th.abs(score_fake)).item()
        im = plt.scatter(var_to_np(inverted.squeeze()[:,0]), var_to_np(inverted.squeeze()[:,1]),cmap=cm.coolwarm,
               c=var_to_np(score_fake), vmin=-max_abs_score, vmax=max_abs_score)
        plt.plot(var_to_np(inputs.squeeze()[:,0]), var_to_np(inputs.squeeze()[:,1]),
                 marker='o', ls='', color=seaborn.color_palette()[2])
        plt.legend(('Real data',))
        cbar = plt.colorbar(im)
        cbar.set_label('Critic Score')
        plt.title("Generated points and critic score")
        display(fig)
        plt.close(fig)
        


In [ ]:
df.loc[:,['sampled_wd', 'wd_dist', 'wd_diff']].plot(figsize=(12,2))
plt.ylim(-5,5)